In [ ]:
!pip install -U accelerate
!pip install --upgrade pip setuptools wheel
!pip install flash-attn --no-build-isolation
!pip install -U deepeval
!pip install openai

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from deepeval.models.base_model import DeepEvalBaseLLM
from openai import OpenAI

In [4]:
class gpt4(DeepEvalBaseLLM):
    def __init__(
        self,
        model,
        tokenizer
    ):
        self.client = OpenAI(
        api_key="sk-or-vv-cbc6ecefae2862b46fc6305099a9185e836364a7d4289be0d6f087bc14b2b210", # ваш ключ в VseGPT после регистрации
        base_url="https://api.vsegpt.ru/v1",
    )

    def load_model(self):
        return self.model

    def generate(self, prompt: str) -> str:
        messages = []
        messages.append({"role": "user", "content": prompt})
        response_big = self.client.chat.completions.create(
            model="openai/gpt-4o-mini", 
            messages=messages,
            temperature=0.2
        )

        #print("Response BIG:",response_big)
        response = response_big.choices[0].message.content
        return response

    async def a_generate(self, prompt: str) -> str:
        return self.generate(prompt)

    def get_model_name(self):
        #ВАЩЕ НЕ МЕНЯЙТЕ!!!!!!!!!!!
        return "Vikhrmodels/Vikhr-Nemo-12B-Instruct-R-21-09-24"

In [5]:
import pandas as pd
train = pd.read_csv('benchmark100.csv')

In [7]:
import json
with open('webhelp_dataset.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

if 'data' in data:
    df = pd.DataFrame(data['data'])

df = df[df.description.str.len() > 4]


df['text'] = df['title'] + ' ' + df['description']

texts_urls = df[['text', 'url']].drop_duplicates(subset='text', keep='first').reset_index(drop=True)
corpus = list(texts_urls.text.values)
urls = texts_urls.url.values

In [8]:

with open('ideal_prompt_t_pro_4bit_contextual.json', 'r') as fl:
    jsoned = json.loads(fl.read())

In [9]:
gptshka = gpt4('', '')

In [10]:
from deepeval.metrics import AnswerRelevancyMetric, FaithfulnessMetric, HallucinationMetric
from deepeval import evaluate
from deepeval.metrics import  ContextualRelevancyMetric, ContextualRecallMetric
from deepeval.test_case import LLMTestCase

ans_metric = AnswerRelevancyMetric(
        threshold=0.7,
        model=gptshka,
        include_reason=False
    )
fai_metric = FaithfulnessMetric(
        threshold=0.7,
        model=gptshka,
        include_reason=False,
        truths_extraction_limit=5
    )

con_metric = ContextualRecallMetric(
    threshold=0.7,
    model=gptshka,
    include_reason=False
)


In [11]:
import copy
def _wrapper(chunks, corpus):
    chunks_ = copy.deepcopy(chunks)
    for j in range(len(chunks)):
        chunks_[j] = f'Чанк {j+1}: {corpus[chunks[j]]}'
    return '\n'.join(chunks_)

In [ ]:
"""

Здесь считаете метрики, но за токенами следите пж

"""
import tqdm
logs_ = []
for i, row in tqdm.tqdm(train.iterrows()):
    actual_output = jsoned[i]['output']
    inp = train.loc[i, 'question']
    expected_output = train.loc[i, 'target']
    retrieval_context = [corpus[j] for j in jsoned[i]['chunks']]
    test_case = LLMTestCase(
        input=inp,
        actual_output=actual_output,
        expected_output=expected_output,
        retrieval_context=retrieval_context
    )
    results = evaluate(
        test_cases=[test_case],
        metrics=[
            ans_metric,
            fai_metric,
            con_metric
        ]
    )
    logs_.append(results)

In [13]:
"""

сейв логов

"""
logs__ = []

for i, lg in enumerate(logs_):
    ttt = []
    for metr in lg.test_results[0].metrics_data:
        ttt.append({
            'MetricData': metr.name,
            'score': metr.score,
            'reason': metr.reason
        })
    logs__.append(ttt)

In [14]:
import json

def save_as_json(filename, data):
    try:
        with open(filename, 'w') as file:
            json.dump(data, file, indent=4) 
        print(f"Data successfully written to {filename}")
    except IOError as e:
        print(f"An error occurred while writing to the file: {e}")

In [15]:
save_as_json('t-pro_contextual_ragas.json', logs__)

Data successfully written to t-pro_contextual_ragas.json
